In [ ]:
import sys ; sys.path.append('../')
from lnos.observer.lueneberger import LuenebergerObserver
from lnos.datasets.exampleSystems import getAutonomousSystem
import lnos.net.linear as ln
from lnos.datasets.plot import plotLogError2D, plotTrajectory2D, plotSimulation2D

import torch
import numpy as np

In [ ]:
# Get plant dynamics
f, h, g, u, dim_x, dim_y, eigen = getAutonomousSystem()

# Initiate observer with system dimensions
observer = LuenebergerObserver(dim_x, dim_y, f, g, h, u)

# Set system dynamics
observer.D = observer.tensorDFromEigen(eigen)
observer.F = torch.tensor([[1.0], [1.0], [1.0]])

In [ ]:
# Create mesh of initial for initial value
sample = np.arange(-1, 1, 0.05)
mesh = np.array(np.meshgrid(sample, sample)).T.reshape(-1, 2)
mesh = torch.tensor(mesh)

In [ ]:
# Generate training data
train_data = ln.generateTrainingData(mesh, observer)

T_star = ln.trainNonlinearLuenbergerTransformation(train_data, observer, False, 20, 2)
T_star = T_star.to("cpu")

In [ ]:
# Simulation parameters for ODE
tsim = (0,50)
dt = 1e-2

# Compute test data
w0_test = torch.tensor([[np.random.uniform(-1,1)], [np.random.uniform(-1,1)],[0],[0],[0]], dtype=torch.double)
tq_test, w_test = observer.simulateLueneberger(w0_test, tsim, dt)

# Data pipeline x_hat
input = w_test.reshape(w_test.shape[0], w_test.shape[1]).float()
input = torch.autograd.Variable(input[:,2:])

# Sample data from T*
x_hat = T_star(input)
x_hat = x_hat.detach().numpy()

In [ ]:
plotSimulation2D(tq_test, np.concatenate((x_hat,w_test[:,:2,0]),axis=1))

In [ ]:
plotTrajectory2D(x_hat.T)

In [ ]:
plotLogError2D(mesh, observer, T_star)